# Wetterdaten von 1950 bis (fast) heute: Vorbereiten

In [ ]:
%matplotlib inline

import io
import pathlib
import urllib
import zipfile

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In dieser Aufgabe verarbeiten, bereinigen und visualisieren Sie Wetterdaten des Deutschen Wetterdienstes. Der Deutsche Wetterdienst stellt für seine zahlreichen Wetterstationen `zip`-Archive mit ihren Messdaten bereit. Dabei gibt es Archive für verschiedene Häufigkeiten (von zehnminütlich bis zum zehnjährigen Mittel), verschiedene Messgrößen (etwa Sonneneinstrahlung oder Wind), sowie historische oder aktuelle Werte. In dieser Aufgabe interessieren wir uns für die historischen Werte (von 1950 bis 2022) der Temperatur und Luftfeuchtigkeit der Station "Gießen/Wettenberg".


Für eine vollständig bepunktete Abgabe sollten Sie bei der Umsetzung insbesondere auf idiomatischen, performanten, und gut lesbaren Code achten:
* Nutzen Sie `pandas`-Methoden, statt explizit zu iterieren.
* Nutzen Sie Method Chaining, statt Rechnungen zu wiederholen oder unnötige Kopien zu erstellen.
* Formatieren Sie Ihren Code sinnvoll, nutzen Sie "sprechende" Variablen und Methoden wo möglich, und kommentieren Sie wo nötig.

## [0 Punkte] Download von den Seiten des Deutschen Wetterdienstes

Die Funktion `extract_zipfile_from_web` lädt eine `.zip`-Datei herunter und extrahiert sie in ein Verzeichnis (standardmäßig `tmp`). Sie gibt eine Liste der extrahierten Dateien zurück.

In [ ]:
def extract_zipfile_from_web(url: str, tmp_dir=pathlib.Path('tmp'), exist_ok=True):
    """Extracts a zipfile from the web to `tmp_dir` and returns list of extracted files.

    url      -- location of the zipfile to download from the web
    tmp_dir  -- output directory for extracted files
    exist_ok -- whether to extract to existing directories

    returns list of paths with extracted files
    """
    tmp_dir.mkdir(exist_ok=exist_ok)

    response = urllib.request.urlopen(url)
    z = zipfile.ZipFile(io.BytesIO(response.read()))

    z.extractall(path=tmp_dir)
    return list(map(pathlib.Path(tmp_dir).joinpath, z.namelist()))

Das gewünschte Archiv (historische, stündliche Messwerte der Temperatur und Luftfeuchte in Gießen/Wettenberg) findet sich unter der folgenden URL:

In [ ]:
url = (
    'https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/hourly/'
    'air_temperature/historical/stundenwerte_TU_01639_19500101_20221231_hist.zip'
)

Laden Sie es zunächst herunter und entpacken Sie es:

In [ ]:
extracted_files = extract_zipfile_from_web(url)
extracted_files

## [1 Punkt] Import des Datensatzes

Das Zip-Archiv enthält einige Dateien mit dem Präfix `Metadaten_`, die Informationen u.A. zu den Messgeräten und zur Geographie enthalten. Die verbleibende Datei mit dem Präfix `produkt_` enthält die tatsächlichen Messwerte. Dabei handelt es sich um eine CSV-artige Datei (lassen Sie nicht von der Endung `.txt` irritieren). Die einzelnen Spalten dieser Datei haben folgende Titel:

```
STATIONS_ID;MESS_DATUM;QN_9;TT_TU;RF_TU;eor
```

`MESS_DATUM` enthält das Messdatum, `TT_TU` die Temperatur in Grad Celsius, und `RF_TU` die relative Luftfeuchtigkeit.

Lesen Sie (ausschließlich) die Spalten `"MESS_DATUM"`, `"TT_TU"` und `"RF_TU"` in einen `DataFrame` ein. Verzichten Sie zunächst auf die Konversion von Datentypen und das Umbenennen von Spalten.

In [ ]:
# Bitte geben Sie Ihren Code hier ein.

## [4 Punkte] Datensatz aufbereiten

Nach dem Einlesen eines Datensatzes muss dieser häufig aufbereitet werden. Dafür bietet es sich an, ein "Rezept" zu schreiben, dessen einzelne Schritte Sie dann nacheinander implementieren können. Hier verwenden wir dieses "Rezept":

1. Datentypen umwandeln
2. Spalten umbenennen
3. Index wählen

#### Datentypen umwandeln
Die Spalte `MESS_DATUM` enthält das Datum einer jeden Messung. Allerdings hat die Spalte nach Import den Typ `int64` und soll ein Datum im Format `%Y%m%d%H` ("JahrMonatTagStunde") repräsentieren. Das ist nicht in unserem Sinne. Stattdessen würden wir das Datum der Messung lieber mit dem Datentyp `datetime64[ns]` speichern. Damit lassen sich die Daten besser verarbeiten und visualisieren.

Dazu könnten Sie die `.astype`, `.to_datetime`, und `.assign`-Methoden nutzen. Beispielsweise lässt sich eine Series von Jahreszahlen als Integer wie folgt in den Typ `datetime64` umwandeln:

```python
>>> years = pd.Series([2019, 2020])
>>> pd.to_datetime(years.astype(str), format='%Y')
pd.Series(['2019-01-01', '2020-01-01'], dtype=datetime64[ns])
```

Wählen Sie außerdem geeignete Datentypen für Temperatur und Luftfeuchtigkeit.

#### Spalten umbenennen
Benennen Sie die Spalten `MESS_DATUM`, `TT_TU`, und `RF_TU` respektive in `Datum`, `Temperatur`, und `Luftfeuchte` um. Dazu bietet sich die `.rename`-Methode an.

#### Index wählen
Beim Einlesen der Daten hat pandas einen neuen Integer-Index erzeugt. Stattdessen wollen wir das `Datum` als Index wählen. Nutzen Sie dazu die `.set_index`-Methode.

In [ ]:
# Bitte geben Sie Ihren Code hier ein.

## [5 Punkte] Datensatz bereinigen

Bei Messungen über lange Zeiträume hinweg kann es durchaus sein, dass einzelne Messungen fehlgeschlagen sind, etwa weil ein Messgerät eine Fehlfunktion hat oder gewartet wird. Zum Teil stehen dann im Datensatz unsinnige Messwerte. Solche Daten müssen detektiert und aus dem Datensatz entfernt werden — ein Problem, das auch allgemein bei großen Datenmengen auftreten kann.

Ein "Rezept" zum Finden unsinniger Messwerten kann folgende Schritte beinhalten:

* Überlegen Sie sich zunächst, welchen Wertebereich Sie (grob) erwarten.
* Prüfen Sie, ob Ihr Datensatz bereits fehlende Werte enthält
* Wiederholen Sie dann die folgenden Schritte, bis Sie keine unsinnigen Werte mehr finden:
    1. Prüfen Sie (besonders) Maximum, Minimum, und Median mit der `.describe`-Methode.
    2. Erstellen Sie ein Histogramm für jede Messgröße.
    3. Suchen Sie in den `.value_counts` nach Werten, die besonders häufig oder besonders selten vorkommen.
    4. Ersetzen Sie alle unsinnigen Messwerte mit einem fehlenden Wert (je nach Datentyp `np.nan`, `pd.NA`, `np.datetime64('NaT')` oder `None`)
    
Wenn Werte fehlen oder entfernt werden mussten, können zum Umgang damit mehrere Strategien sinnvoll sein:
* im Datensatz belassen
* aus dem Datensatz entfernen
* interpolieren, z.B. linear, mit Splines, oder dem nächsten bekannten Wert
* mit einem sinnvollen Wert ersetzen, z.B. Mittelwert oder Median

Überprüfen Sie auf geeignete Weise, ob und welche Messwerte für Temperatur und/oder Luftfeuchtigkeit in einem nicht sinnvollen Bereich liegen, und entfernen Sie diese. Achten Sie darauf, keine unnötigen Kopien der Daten zu erzeugen, sondern den existierenden `DataFrame` geeignet anzupassen. Dokumentieren Sie Ihr Vorgehen angemessen.

In [ ]:
# Bitte geben Sie Ihren Code hier ein.